### Imports

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import json
from tqdm._tqdm_notebook import tqdm_notebook
import itertools
from time import time

warnings.filterwarnings("ignore")

<ipython-input-1-e77114c5ecb0>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook


### Read Data

In [2]:
data = pd.read_csv('myFitnessPal_parsed.csv')
data.head()

,user_id,sequence,food_ids,total_calories,total_carbs,total_fat,total_protein,total_sodium,total_sugar,goal_calories,...,year,month,day,foods_len,calories_diff,carbs_diff,fat_diff,protein_diff,sodium_diff,sugar_diff
0,1,1,"[1, 2, 3, 4, 4]",2430,96,37.0,50.0,855.0,63.0,1572.0,...,2014,9,15,15,-858.0,100.0,15.0,29.0,1445.0,-4.0
1,1,1,"[5, 1, 2, 3, 6, 7]",1862,158,54.0,114.0,2215.0,100.0,1832.0,...,2014,9,16,18,-30.0,71.0,7.0,-22.0,85.0,-31.0
2,1,1,"[1, 2, 3, 6, 8, 9, 10]",2251,187,60.0,98.0,1765.0,105.0,1685.0,...,2014,9,17,22,-566.0,23.0,-4.0,-13.0,535.0,-42.0
3,1,1,"[1, 6, 2, 3, 11, 12]",2001,113,81.0,202.0,1101.0,71.0,1597.0,...,2014,9,18,20,-404.0,86.0,-28.0,-122.0,1199.0,-11.0
4,1,1,"[1, 7, 13, 12, 2, 3, 12, 12]",2158,180,89.0,115.0,1998.0,84.0,1589.0,...,2014,9,19,28,-569.0,18.0,-36.0,-35.0,302.0,-24.0


In [3]:
with open('foods.json') as json_file:
     foods = json.load(json_file)

In [4]:
# there are 145 people which logged only one day.
quantity = data["user_id"].value_counts()
data["raw_quantity"] = data["user_id"].apply(lambda x: quantity[x])
data[data["raw_quantity"]<2]["user_id"].value_counts()

8185    1
1376    1
8481    1
4431    1
6482    1
       ..
680     1
3754    1
686     1
5808    1
1024    1
Name: user_id, Length: 401, dtype: int64

In [6]:
def count_freq(row, n):
    for subset in set(itertools.combinations(row[1:-1].split(', ') , n)):
        if len(set(subset)) == n:
            food_combos[subset] = food_combos.get(subset, 0) + 1

In [7]:
# 4 Meals Combo
start = time()
food_combos = {}
data.food_ids.apply(lambda x: count_freq(x, 4))
temp = sorted(food_combos.items(), key=lambda item: item[1])
temp.reverse()
a = pd.DataFrame(temp[:10], columns = ['Combo', 'Amount'])

a['Meal_1_ID'] = a['Combo'].apply(lambda x: x[0])
a['Meal_1_Name'] = a['Combo'].apply(lambda x: foods[x[0]])

a['Meal_2_ID'] = a['Combo'].apply(lambda x: x[1])
a['Meal_2_Name'] = a['Combo'].apply(lambda x: foods[x[1]])

a['Meal_3_ID'] = a['Combo'].apply(lambda x: x[2])
a['Meal_3_Name'] = a['Combo'].apply(lambda x: foods[x[2]])

a['Meal_4_ID'] = a['Combo'].apply(lambda x: x[3])
a['Meal_4_Name'] = a['Combo'].apply(lambda x: foods[x[3]])

a[['Meal_1_ID','Meal_1_Name','Meal_2_ID','Meal_2_Name','Meal_3_ID','Meal_3_Name','Meal_4_ID','Meal_4_Name','Amount']].to_csv('./4 Meals Combo.csv', index=False)

print(time()-start)
a[['Meal_1_ID','Meal_1_Name','Meal_2_ID','Meal_2_Name','Meal_3_ID','Meal_3_Name','Meal_4_ID','Meal_4_Name','Amount']].head()

KeyboardInterrupt: 

In [ ]:
# 3 Meals Combo
start = time()
food_combos = {}
data.food_ids.apply(lambda x: count_freq(x, 3))
temp = sorted(food_combos.items(), key=lambda item: item[1])
temp.reverse()
a = pd.DataFrame(temp[:10], columns = ['Combo', 'Amount'])

a['Meal_1_ID'] = a['Combo'].apply(lambda x: x[0])
a['Meal_1_Name'] = a['Combo'].apply(lambda x: foods[x[0]])

a['Meal_2_ID'] = a['Combo'].apply(lambda x: x[1])
a['Meal_2_Name'] = a['Combo'].apply(lambda x: foods[x[1]])

a['Meal_3_ID'] = a['Combo'].apply(lambda x: x[2])
a['Meal_3_Name'] = a['Combo'].apply(lambda x: foods[x[2]])

a[['Meal_1_ID','Meal_1_Name','Meal_2_ID','Meal_2_Name','Meal_3_ID','Meal_3_Name','Amount']].to_csv('./3 Meals Combo.csv', index=False)

print(time()-start)
a[['Meal_1_ID','Meal_1_Name','Meal_2_ID','Meal_2_Name','Meal_3_ID','Meal_3_Name','Amount']].head()

### Which foods are eaten most often in a month

In [ ]:
def add_elem_dict(all_names,lst):
    for food_id in lst[1:-1].split(","):
        food_id = food_id.replace(' ','')
        all_names[food_id] = all_names.get(food_id,0)+1
        
def get_top_food_ids(df,top_numbers):
    all_names = {}
    df["food_ids"].apply(lambda lst: add_elem_dict(all_names,lst))
    top_food_ids = list(sorted(all_names.items(), key=lambda item: item[1]))
    top_food_ids = [key for key,value in top_food_ids]
    top_food_ids.reverse()
    return top_food_ids[:10]

In [ ]:
max_food_in_month = data.groupby(['month']).apply(lambda x: get_top_food_ids(x,10))

In [ ]:
top_food_dataframe = pd.DataFrame(max_food_in_month)

top_food_dataframe = top_food_dataframe.apply(lambda x: x[0], axis = 1, result_type ='expand').reset_index()
top_food_dataframe.columns = ["month","meal_1","meal_2","meal_3","meal_4","meal_5","meal_6","meal_7","meal_8","meal_9","meal_10"]

In [ ]:
# თუ უკეთ იცი დაწერა დაწერე 
top_food_dataframe["meal_1"] = top_food_dataframe["meal_1"].apply(lambda food_id: foods[str(food_id)])
top_food_dataframe["meal_2"] = top_food_dataframe["meal_2"].apply(lambda food_id: foods[str(food_id)])
top_food_dataframe["meal_3"] = top_food_dataframe["meal_3"].apply(lambda food_id: foods[str(food_id)])
top_food_dataframe["meal_4"] = top_food_dataframe["meal_4"].apply(lambda food_id: foods[str(food_id)])
top_food_dataframe["meal_5"] = top_food_dataframe["meal_5"].apply(lambda food_id: foods[str(food_id)])
top_food_dataframe["meal_6"] = top_food_dataframe["meal_6"].apply(lambda food_id: foods[str(food_id)])
top_food_dataframe["meal_7"] = top_food_dataframe["meal_7"].apply(lambda food_id: foods[str(food_id)])
top_food_dataframe["meal_8"] = top_food_dataframe["meal_8"].apply(lambda food_id: foods[str(food_id)])
top_food_dataframe["meal_9"] = top_food_dataframe["meal_9"].apply(lambda food_id: foods[str(food_id)])
top_food_dataframe["meal_10"] = top_food_dataframe["meal_10"].apply(lambda food_id: foods[str(food_id)])

In [ ]:
top_food_dataframe

### Square Root Residuals of calories mismatch between goal and actual

In [ ]:
data['SquareRootResiduals'] = (data['total_calories']-data['goal_calories']).apply(lambda x: np.sqrt(x**2))
data.head()

### Find applicants who have SquareRootResiduals greater than threshold and plot without them

In [ ]:
data_unique_ID_date = data.drop_duplicates(subset=['user_id', 'month', 'day', 'year'])
SquareRootResiduals_threshold = 10_000
applicants_with_high_SquareRootResiduals = data_unique_ID_date[data_unique_ID_date["SquareRootResiduals"]>=SquareRootResiduals_threshold]

applicants_with_high_SquareRootResiduals.sort_values(by='SquareRootResiduals', ascending=False).head()

In [ ]:
fig, ax =plt.subplots(1,1,figsize=(20,10))

sns.distplot(data_unique_ID_date[data_unique_ID_date["SquareRootResiduals"]<SquareRootResiduals_threshold]["SquareRootResiduals"], bins=100, ax=ax)

fig.show()

### Find top users

In [ ]:
user_tenure = data['user_id'].value_counts().to_frame().reset_index()
user_tenure.columns = ['user_id', 'Amount']

top_users = pd.merge(data,user_tenure.head(1000),on='user_id', how='inner')
top_users.head()

In [ ]:
def enum_date(UserID):
    global user_id, indx
    if UserID != user_id:
        user_id = UserID
        indx = 0
    
    indx += 1
    return indx

In [ ]:
user_id = 1
indx = 0

top_users['eating_enum'] = top_users['user_id'].apply(lambda x: enum_date(x))

In [ ]:
avg = top_users.groupby(['user_id']).apply(lambda x: len(x))

avg.sum()/len(avg)

### Plot Nutritient separately

In [ ]:
def plot_nutrient(df,nutrient_name,goal_nutrient_name,thr1,thr2):
    avg_nutrient = df.groupby(['user_id']).apply(lambda x: x[nutrient_name].sum()/len(x)).reset_index()
    goal_avg_nutrient = df.groupby(['user_id']).apply(lambda x: x[goal_nutrient_name].sum()/len(x)).reset_index()
    avg_nutrient.columns = ["user_id", "average_nutrient_calories"]
    goal_avg_nutrient.columns = ["user_id", "goal_average_nutrient_calories"]
    
    fig, ax =plt.subplots(2,3,figsize=(26,10))

    sns.distplot(avg_nutrient[avg_nutrient['average_nutrient_calories'] < thr1]['average_nutrient_calories'], ax=ax[0][0]).set_title("Lower risk")
    sns.distplot(avg_nutrient['average_nutrient_calories'], ax=ax[0][1]).set_title("General")
    sns.distplot(avg_nutrient[avg_nutrient['average_nutrient_calories'] > thr2]['average_nutrient_calories'], ax=ax[0][2]).set_title("Above risk")
    
    sns.distplot(goal_avg_nutrient[goal_avg_nutrient['goal_average_nutrient_calories'] < thr1]['goal_average_nutrient_calories'], ax=ax[1][0])
    sns.distplot(goal_avg_nutrient['goal_average_nutrient_calories'], ax=ax[1][1])
    sns.distplot(goal_avg_nutrient[goal_avg_nutrient['goal_average_nutrient_calories'] > thr2]['goal_average_nutrient_calories'], ax=ax[1][2])
    
    fig.show()
    
    under_minimum = len(avg_nutrient[avg_nutrient['average_nutrient_calories'] < thr1])
    above_minimum = len(avg_nutrient[avg_nutrient['average_nutrient_calories'] > thr2])
    total = avg_nutrient.user_id.nunique()

    print(f'\nlower risk group: {100*under_minimum/total} %')
    print(f'above risk group: {100*above_minimum/total} %')
    print(f'not   risk group: {100*(1-(under_minimum+above_minimum)/total)} %\n')

In [ ]:
plot_nutrient(top_users,"total_carbs","goal_carbs",50,350)

In [ ]:
plot_nutrient(top_users,"total_fat","goal_fat",50,350)

In [ ]:
plot_nutrient(top_users,"total_protein","goal_protein",50,350)

In [ ]:
plot_nutrient(top_users,"total_sodium","goal_sodium",50,350)

In [ ]:
plot_nutrient(top_users,"total_sugar","goal_sugar",50,350)